In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
findspark.find()

'/Users/avinashs/Downloads/softwares/spark-3.0.0-bin-hadoop2.7'

In [4]:
from pyspark.sql import SparkSession

### Spark Session object

In [6]:
spark = SparkSession.builder.appName("Anaconda python").master("local[5]").getOrCreate()

22/08/13 07:26:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
spark

### Create Spark Context from Spark session object

In [8]:
sc = spark.sparkContext

### Create rdd from memory

In [ ]:
numbers_rdd = sc.parallelize(range(1,101), 4)

In [ ]:
numbers_rdd

## Map partitions with Index

In [ ]:
def print_elements_in_partition(index, elements):
    for element in elements:
        yield f"partition: {index} has element: {element}"

In [ ]:
numbers_rdd.mapPartitionsWithIndex(print_elements_in_partition).collect()

### Map transfromation

In [ ]:
numbers_rdd.map(lambda x: x*x).collect()

### Filter transformation

In [ ]:
numbers_rdd.filter(lambda x : x%2 ==0).collect()

### Get Number of partitions of a rdd

In [ ]:
numbers_rdd.getNumPartitions()

### Count Action

In [ ]:
numbers_rdd.count()

### flatMap transformation

In [ ]:
sentences = ["This is a sentence1.", "This is sentence2.", "This is sentence3"]

In [ ]:
sentence_rdd = sc.parallelize(sentences)

In [ ]:
sentence_rdd.collect()

In [ ]:
sentence_rdd.flatMap(lambda x:x.split()).collect()

In [ ]:
rdd = sc.textFile("/Users/avinashs/Downloads/pyspark/datasets/Baby_Names__Beginning_2007.csv")

In [ ]:
rdd

In [ ]:
rdd.getNumPartitions()

In [ ]:
rdd.collect()

In [ ]:
rdd.count()

In [ ]:
def count_baby_names_partitions_wise(index, iterator):
    count_records = 0
    for row in iterator:
        count_records += 1
    yield f"Parition {index} has {count_records} records"

In [ ]:
rdd.mapPartitionsWithIndex(count_baby_names_partitions_wise).collect()

In [ ]:
def skip_header(index, iterator):
    number = 0
    for row in iterator:
        if index == 0 and number == 0:
            number += 1
            continue
        yield row
            
    
            

In [ ]:
data_rdd = rdd.mapPartitionsWithIndex(skip_header)

In [ ]:
year_rdd = data_rdd.map(lambda x: (x.split(",")[0], int(x.split(",")[-1])))

In [ ]:
year_rdd.reduceByKey(lambda x, y : x+y).collect()

In [ ]:
year_rdd.groupByKey().mapValues(sum).collect()

In [ ]:
year_rdd.getNumPartitions()

### Increase the nuber of partition

In [ ]:
year_rdd = year_rdd.repartition(4)

In [ ]:
year_rdd.getNumPartitions()

### Decrease the number of partition

In [ ]:
year_rdd.coalesce(3).getNumPartitions()

### Write rdd to a text file

In [ ]:
year_rdd.saveAsTextFile("years.txt")

In [ ]:
rdd_repartition = rdd.repartition(4)

In [ ]:
rdd_repartition.collect()

In [9]:
file_rdd = sc.textFile("/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/ebook.txt", 4)

In [ ]:
file_rdd.collect()

In [ ]:
file_rdd.flatMap(lambda x: x.split(" ")).map(lambda x : (x,1)).reduceByKey(lambda x,y: x+y).collect()

In [10]:
filename = "/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/weather.csv"
file1_rdd = sc.textFile(filename)

### take action

In [11]:
file1_rdd.take(5)

['2016-05-09,234893,34',
 '2019-09-08,234896,3',
 '2019-11-19,234895,24',
 '2017-04-04,234900,43',
 '2013-12-04,234900,47']

In [12]:
file1_rdd.first()

'2016-05-09,234893,34'

In [15]:
file1_rdd.map(lambda x: (x.split(",")[0], int(x.split(",")[-1]))).map(lambda x: (int(x[0].split("-")[0]), x[1])).reduceByKey(lambda x, y: max(x,y)).sortByKey().collect()

[(2010, 39),
 (2011, 38),
 (2012, 40),
 (2013, 47),
 (2014, 35),
 (2015, 41),
 (2016, 36),
 (2017, 47),
 (2018, 45),
 (2019, 47)]

### Accumulators

In [16]:
accum = sc.accumulator(0)

In [17]:
file_rdd = sc.textFile("/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/ebook.txt")

In [19]:
file_rdd.getNumPartitions()

2

In [35]:
def count_blank_lines_in_partition(iterator):
    count_blank_lines = 0
    for row in iterator:
        if len(row.strip()) == 0:
            count_blank_lines += 1

    accum.add(count_blank_lines)

In [37]:
file_rdd.foreachPartition(count_blank_lines_in_partition)

In [38]:
accum.value

539

22/08/13 13:07:31 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 7200643 ms exceeds timeout 120000 ms
22/08/13 13:07:31 WARN SparkContext: Killing executors is not supported by current scheduler.


### read json file in python

In [55]:
import json
from pprint import pprint

filename = "/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/people.json"
texts = []
for line in open(filename, 'r'):
    texts.append(json.loads(line))

In [57]:
json_rdd = sc.textFile(filename)

In [79]:
def convert_to_csv(record):
    return tuple(json.loads(record).values())

In [81]:
json_rdd.map(convert_to_csv).saveAsTextFile("json_csv")

In [125]:
rdd = sc.textFile("/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/Baby_Names__Beginning_2007.csv")

In [126]:
rdd.take(5)

['Year,First Name,County,Sex,Count',
 '2013,GAVIN,ST LAWRENCE,M,9',
 '2013,LEVI,ST LAWRENCE,M,9',
 '2013,LOGAN,NEW YORK,M,44',
 '2013,HUDSON,NEW YORK,M,49']

In [127]:
first_rdd = rdd.map(lambda x : (x.split(",")[0], x.split(",")[1]))

In [128]:
second_rdd = rdd.map(lambda x: (x.split(",")[1], x.split(",")[-2]))

In [129]:
rdd.persist()

/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/Baby_Names__Beginning_2007.csv MapPartitionsRDD[85] at textFile at NativeMethodAccessorImpl.java:0

In [130]:
first_rdd.collect()

[('Year', 'First Name'),
 ('2013', 'GAVIN'),
 ('2013', 'LEVI'),
 ('2013', 'LOGAN'),
 ('2013', 'HUDSON'),
 ('2013', 'GABRIEL'),
 ('2013', 'THEODORE'),
 ('2013', 'ELIZA'),
 ('2013', 'MADELEINE'),
 ('2013', 'ZARA'),
 ('2013', 'DAISY'),
 ('2013', 'JONATHAN'),
 ('2013', 'CHRISTOPHER'),
 ('2013', 'LUKE'),
 ('2013', 'JACKSON'),
 ('2013', 'JACKSON'),
 ('2013', 'JOSHUA'),
 ('2013', 'AIDEN'),
 ('2013', 'BRANDON'),
 ('2013', 'JUDY'),
 ('2013', 'MASON'),
 ('2013', 'DAVID'),
 ('2013', 'NOAH'),
 ('2013', 'AIDEN'),
 ('2013', 'SEBASTIAN'),
 ('2013', 'SAMUEL'),
 ('2013', 'MAX'),
 ('2013', 'DEVORA'),
 ('2013', 'OLIVER'),
 ('2013', 'JOSEPH'),
 ('2013', 'JAYDEN'),
 ('2013', 'LEO'),
 ('2013', 'JOHN'),
 ('2013', 'MICHAEL'),
 ('2013', 'MASON'),
 ('2013', 'MATTHEW'),
 ('2013', 'CHARLES'),
 ('2013', 'YEHUDIS'),
 ('2013', 'SABRINA'),
 ('2013', 'LUNA'),
 ('2013', 'MILAN'),
 ('2013', 'DANIELLE'),
 ('2013', 'ISLA'),
 ('2013', 'PARIS'),
 ('2013', 'LOLA'),
 ('2013', 'NYLAH'),
 ('2013', 'HELEN'),
 ('2013', 'ADELE'),


In [131]:
second_rdd.collect()

[('First Name', 'Sex'),
 ('GAVIN', 'M'),
 ('LEVI', 'M'),
 ('LOGAN', 'M'),
 ('HUDSON', 'M'),
 ('GABRIEL', 'M'),
 ('THEODORE', 'M'),
 ('ELIZA', 'F'),
 ('MADELEINE', 'F'),
 ('ZARA', 'F'),
 ('DAISY', 'F'),
 ('JONATHAN', 'M'),
 ('CHRISTOPHER', 'M'),
 ('LUKE', 'M'),
 ('JACKSON', 'M'),
 ('JACKSON', 'M'),
 ('JOSHUA', 'M'),
 ('AIDEN', 'M'),
 ('BRANDON', 'M'),
 ('JUDY', 'F'),
 ('MASON', 'M'),
 ('DAVID', 'M'),
 ('NOAH', 'M'),
 ('AIDEN', 'M'),
 ('SEBASTIAN', 'M'),
 ('SAMUEL', 'M'),
 ('MAX', 'M'),
 ('DEVORA', 'F'),
 ('OLIVER', 'M'),
 ('JOSEPH', 'M'),
 ('JAYDEN', 'M'),
 ('LEO', 'M'),
 ('JOHN', 'M'),
 ('MICHAEL', 'M'),
 ('MASON', 'M'),
 ('MATTHEW', 'M'),
 ('CHARLES', 'M'),
 ('YEHUDIS', 'F'),
 ('SABRINA', 'F'),
 ('LUNA', 'F'),
 ('MILAN', 'F'),
 ('DANIELLE', 'F'),
 ('ISLA', 'F'),
 ('PARIS', 'F'),
 ('LOLA', 'F'),
 ('NYLAH', 'F'),
 ('HELEN', 'F'),
 ('ADELE', 'F'),
 ('LIAM', 'M'),
 ('LIAM', 'M'),
 ('DYLAN', 'M'),
 ('DANIEL', 'M'),
 ('JAMES', 'M'),
 ('BENJAMIN', 'M'),
 ('HENRY', 'M'),
 ('RYAN', 'M'),
 ('LU

In [118]:
rdd = rdd.unpersist()

In [138]:
rdd = rdd.unpersist(blocking=False)

In [139]:
rdd.collect()

['Year,First Name,County,Sex,Count',
 '2013,GAVIN,ST LAWRENCE,M,9',
 '2013,LEVI,ST LAWRENCE,M,9',
 '2013,LOGAN,NEW YORK,M,44',
 '2013,HUDSON,NEW YORK,M,49',
 '2013,GABRIEL,NEW YORK,M,50',
 '2013,THEODORE,NEW YORK,M,51',
 '2013,ELIZA,KINGS,F,16',
 '2013,MADELEINE,KINGS,F,16',
 '2013,ZARA,KINGS,F,16',
 '2013,DAISY,KINGS,F,16',
 '2013,JONATHAN,NEW YORK,M,51',
 '2013,CHRISTOPHER,NEW YORK,M,52',
 '2013,LUKE,SUFFOLK,M,49',
 '2013,JACKSON,NEW YORK,M,53',
 '2013,JACKSON,SUFFOLK,M,49',
 '2013,JOSHUA,NEW YORK,M,53',
 '2013,AIDEN,NEW YORK,M,53',
 '2013,BRANDON,SUFFOLK,M,50',
 '2013,JUDY,KINGS,F,16',
 '2013,MASON,ST LAWRENCE,M,8',
 '2013,DAVID,NEW YORK,M,53',
 '2013,NOAH,ST LAWRENCE,M,8',
 '2013,AIDEN,ST LAWRENCE,M,8',
 '2013,SEBASTIAN,NEW YORK,M,57',
 '2013,SAMUEL,NEW YORK,M,59',
 '2013,MAX,NEW YORK,M,62',
 '2013,DEVORA,KINGS,F,16',
 '2013,OLIVER,NEW YORK,M,66',
 '2013,JOSEPH,NEW YORK,M,66',
 '2013,JAYDEN,NEW YORK,M,66',
 '2013,LEO,NEW YORK,M,67',
 '2013,JOHN,NEW YORK,M,68',
 '2013,MICHAEL,NEW YO

In [140]:
help(rdd.cache)

Help on method cache in module pyspark.rdd:

cache() method of pyspark.rdd.RDD instance
    Persist this RDD with the default storage level (`MEMORY_ONLY`).



In [141]:
help(rdd.persist)

Help on method persist in module pyspark.rdd:

persist(storageLevel=StorageLevel(False, True, False, False, 1)) method of pyspark.rdd.RDD instance
    Set this RDD's storage level to persist its values across operations
    after the first time it is computed. This can only be used to assign
    a new storage level if the RDD does not have a storage level set yet.
    If no storage level is specified defaults to (`MEMORY_ONLY`).
    
    >>> rdd = sc.parallelize(["b", "a", "c"])
    >>> rdd.persist().is_cached
    True



In [146]:
rdd.is_cached

False

In [145]:
rdd.unpersist()

/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/Baby_Names__Beginning_2007.csv MapPartitionsRDD[85] at textFile at NativeMethodAccessorImpl.java:0

### To see all the spark configuration variables

In [157]:
sc._conf.getAll()

[('spark.driver.port', '61696'),
 ('spark.driver.host', '192.168.0.104'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1660355818904'),
 ('spark.master', 'local[5]'),
 ('spark.app.name', 'Anaconda python'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [158]:
rdd.getNumPartitions()

2

In [160]:
rdd.foreachPartition(print)

<generator object UTF8Deserializer.load_stream at 0x7f8e280ab350>
<generator object UTF8Deserializer.load_stream at 0x7f8e280ab350>


In [161]:
spark.stop()